In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import os

In [ ]:
stroke_registry_data_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/stroke_registry/post_hoc_modified/stroke_registry_post_hoc_modified.xlsx'
patient_selection = '/Users/jk1/temp/opsum_extration_output/high_frequency_data_patient_selection.csv'
out_dir = '/Users/jk1/temp/opsum_extration_output'

In [ ]:
patient_selection_df = pd.read_csv(patient_selection, dtype=str)
full_data_df = pd.read_excel(stroke_registry_data_path)

In [ ]:
full_data_df['patient_id'] = full_data_df['Case ID'].apply(lambda x: x[8:-4])
full_data_df['EDS_last_4_digits'] = full_data_df['Case ID'].apply(lambda x: x[-4:])
full_data_df['case_admission_id'] = full_data_df['patient_id'].astype(str) + full_data_df['EDS_last_4_digits'].astype(str) + '_' + pd.to_datetime(full_data_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d%m%Y').astype(str)
patient_selection_df['case_admission_id'] = patient_selection_df['patient_id'].astype(str) + patient_selection_df['EDS_last_4_digits'].astype(str) + '_' + pd.to_datetime(patient_selection_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d%m%Y').astype(str)

In [ ]:
selected_full_data_df = full_data_df[
    full_data_df['case_admission_id'].isin(patient_selection_df['case_admission_id'].tolist())]

selected_full_data_df['begin_date'] = pd.to_datetime(selected_full_data_df['Arrival at hospital'],
                                                     format='%Y%m%d').dt.strftime('%d.%m.%Y') + ' ' + selected_full_data_df['Arrival time']

In [ ]:
selected_full_data_df['onset_datetime'] = pd.to_datetime(pd.to_datetime(selected_full_data_df['Onset date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + selected_full_data_df['Onset time'], format='%d-%m-%Y %H:%M')
selected_full_data_df['imaging_datetime'] = pd.to_datetime(pd.to_datetime(selected_full_data_df['1st brain imaging date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + selected_full_data_df['1st brain imaging time'], format='%d-%m-%Y %H:%M')

In [ ]:
selected_full_data_df['begin_date']

In [ ]:
selected_full_data_df['onset_to_imaging_min'] = (pd.to_datetime(selected_full_data_df['imaging_datetime'], format='%d-%m-%Y %H:%M:%S') - pd.to_datetime(selected_full_data_df['onset_datetime'], format='%d-%m-%Y %H:%M:%S')).dt.total_seconds() / 60
selected_full_data_df['door_to_image_min'] = (pd.to_datetime(selected_full_data_df['imaging_datetime'], format='%d-%m-%Y %H:%M:%S') - pd.to_datetime(selected_full_data_df['begin_date'], format='%d.%m.%Y %H:%M')).dt.total_seconds() / 60
selected_full_data_df['onset_to_admission_min'] = (pd.to_datetime(selected_full_data_df['begin_date'], format='%d.%m.%Y %H:%M') - pd.to_datetime(selected_full_data_df['onset_datetime'], format='%d-%m-%Y %H:%M:%S')).dt.total_seconds() / 60

In [ ]:
ax = selected_full_data_df.plot.scatter(x='onset_to_imaging_min', y='onset_to_admission_min', color='blue')
selected_full_data_df.plot.scatter(x='onset_to_imaging_min', y='door_to_image_min', color='red', ax=ax)
ax.set_xlim([0, 1880])
ax.set_ylim([0, 1880])
plt.show()


In [ ]:
selected_full_data_df['onset_to_imaging_min'].isna().sum()

In [ ]:
selected_full_data_df[['onset_datetime', 'imaging_datetime', 'onset_to_admission_min', 'onset_to_imaging_min', 'door_to_image_min', 'Door to image (min.)', '1st brain imaging type']]

In [ ]:
selected_full_data_df[selected_full_data_df['onset_to_admission_min'] > 0]['onset_to_admission_min'].quantile(0.05)

In [ ]:
selected_full_data_df[selected_full_data_df['onset_to_admission_min'] > 0]['onset_to_admission_min'].quantile(0.95)

In [ ]:
selected_full_data_df['Time of symptom onset known'].value_counts()

In [ ]:
temp_df = selected_full_data_df[(selected_full_data_df['Referral'] != 'In-hospital event') & ((selected_full_data_df["onset_to_admission_min"] > 96 * 60) | (selected_full_data_df["onset_to_admission_min"] < 0)) & (selected_full_data_df['Time of symptom onset known'] != 'no')]
temp_df[['Case ID', 'onset_to_admission_min', 'onset_datetime', 'begin_date', 'imaging_datetime', 'Referral']]

In [ ]:
# temp_df.to_excel(os.path.join(out_dir, 'onset_to_admission_min_outlier.xlsx'))

In [ ]:
temp_df = selected_full_data_df[(selected_full_data_df['Referral'] != 'In-hospital event') & (selected_full_data_df['Time of symptom onset known'] != 'no') & (selected_full_data_df["onset_to_admission_min"] < 96 * 60)]
ax = temp_df['onset_to_admission_min'].plot.hist(bins=100)
ax.set_xlim([0, 2880])
plt.show()

## Categorize onset to admission timings

Categories
- onset_unknown
- intra_hospital
- <270min (4.5h)
- 271-540min (9h)
- 541-1440min (24h)
- \>1440min

In [ ]:
selected_full_data_df['categorical_onset_to_admission_time'] = pd.cut(selected_full_data_df['onset_to_admission_min'],
                                                  bins=[-float("inf"), 270, 540,1440,float("inf")],
                                                  labels=['<270min', '271-540min', '541-1440min', '>1440min'])

In [ ]:
selected_full_data_df['categorical_onset_to_admission_time'] = selected_full_data_df['categorical_onset_to_admission_time'].cat.add_categories('intra_hospital')
selected_full_data_df['categorical_onset_to_admission_time'] = selected_full_data_df['categorical_onset_to_admission_time'].cat.add_categories('onset_unknown')

In [ ]:
selected_full_data_df.loc[selected_full_data_df['Referral'] == 'In-hospital event', 'categorical_onset_to_admission_time'] = 'intra_hospital'

In [ ]:
selected_full_data_df.loc[selected_full_data_df['Time of symptom onset known'] == 'no', 'categorical_onset_to_admission_time'] = 'onset_unknown'

In [ ]:
selected_full_data_df[['categorical_onset_to_admission_time', 'onset_to_admission_min']]

In [ ]:
selected_full_data_df['categorical_onset_to_admission_time'].value_counts()

In [ ]:
selected_full_data_df['Time of symptom onset known'].value_counts()

In [ ]:
selected_full_data_df['wake_up_stroke'] = selected_full_data_df['Time of symptom onset known'].apply(lambda x: True if x == 'wake up'else False)

In [ ]:
timing_columns = ['categorical_onset_to_admission_time', 'wake_up_stroke']

In [ ]:
timing_df = selected_full_data_df[timing_columns + ['case_admission_id', 'begin_date']]

In [ ]:
timing_df.isna().sum()

In [ ]:
timing_df = pd.melt(timing_df, id_vars=['case_admission_id', 'begin_date'], var_name='sample_label')
timing_df

In [ ]:
from preprocessing.stroke_registry_params_preprocessing.timing_params_preprocessing import preprocess_timing_params

timing_bis = preprocess_timing_params(selected_full_data_df)

In [ ]:
(timing_df == timing_bis).all()